In [1]:
import cv2 as cv

from matplotlib import pyplot as plt
import networkx as nx
import itertools
import numpy as np
import math

In [2]:
nonoise_vid = cv.VideoCapture('./1tagvideo.mp4')
if (nonoise_vid.isOpened() == False):
	print("Error opening the video file")
else:
  # Get frame rate information

  fps = int(nonoise_vid.get(5))
  print("Frame Rate : ",fps,"frames per second")	

  # Get frame count
  frame_count = nonoise_vid.get(7)
  print("Frame count : ", frame_count)


Frame Rate :  26 frames per second
Frame count :  787.0


In [3]:
frames = []
while(nonoise_vid.isOpened()):
	# nonoise_vid.read() methods returns a tuple, first element is a bool 
	# and the second is frame
  ret, frame = nonoise_vid.read()
  if ret == True:
    #Split the channels and invert color of the frame as we are only interested in the red channel
    frames.append(frame)
    break
  else:
    break

# SCRATHPAD

In [4]:
def detect_rectangles(corners:list()):
    list_of_point_pairs = []
    for point_pairs in itertools.combinations(list_of_corners,2):
        p1,p2= point_pairs[0],point_pairs[1]
        dist = np.linalg.norm(p1-p2)
        if(dist>250):
            slope = (p2[1]-p1[1])/(p2[0]-p1[0])
            center =  np.array([(p1[0]+p2[0])/2,(p1[1]+p2[1])/2])
            list_of_point_pairs.append([p1,p2,slope,center,dist])
               
    array_of_corners = np.array(list_of_point_pairs,dtype=object)
    array_of_corners = array_of_corners[np.argsort(array_of_corners[:,-1])][::-1]
    
    list_of_rectangles = []
    list_of_center_dist = []
    for line_pairs in itertools.combinations(array_of_corners,2):
        n,m = line_pairs[0],line_pairs[1]
        if(np.abs(n[-1]-m[-1])>5):
            continue
        if (np.linalg.norm(m[3]-n[3])>1):
            continue
        list_of_rectangles.append(np.array([tuple(n[0].tolist()),tuple(m[0].tolist()),tuple(n[1].tolist()),tuple(m[1].tolist())]))
        list_of_center_dist.append(np.array([n[3],np.linalg.norm(n[0]-m[0])],dtype = object))
    return list_of_rectangles,list_of_center_dist

In [5]:
def plot_polygon(img, lov):
    for i in lov:
        pts = np.array(i, np.int32)
        pts = pts.reshape((-1,1,2))
        img_rect = cv.polylines(img,[pts],True,(0,255,255))
    return img_rect

In [6]:
def calculate_homography(world_corners,image_corner):
    Xw = world_corners
    xc = image_corner
    matrix_A = np.matrix(np.zeros((8,9)))
    for a,b in zip(enumerate(Xw),enumerate(xc)):
        i,j,n,m = a[0],b[0],a[1],b[1]
        matrix_A[i+j,:] = -n[0],-n[1],-1,0,0,0,m[0]*n[0],m[0]*n[1],m[0]
        matrix_A[(i+j)+1,:] = 0,0,0,-n[0],-n[1],-1,m[1]*n[0],m[1]*n[1],m[1]
    U,S,V = np.linalg.svd(matrix_A)
    H = np.reshape(V[-1],[3,3])
    return H/H[-1,-1]

In [7]:
def decode_tag(tag):
    tag = cv.cvtColor(tag,cv.COLOR_BGR2GRAY)
    _,tag = cv.threshold(tag,127,255,cv.THRESH_OTSU+cv.THRESH_OTSU)
    tag_grids = np.array(np.array_split(tag,8))
    msg = np.zeros((8,8))
    for i,tag_grid in enumerate(tag_grids):
        for j,grid in  enumerate(np.array_split(tag_grid,8,axis=1)):
            if np.count_nonzero(grid) < 0.5*grid.size:
                msg[i][j] = 0
            else:
                msg[i][j] =1
    if msg[2][2]:
        ori = 180
    elif msg[2][5]:
        ori = 90
    elif msg[5][2]:
        ori = 270
    elif msg[5][5]:
        ori = 0
    else :
        ori = None
    idx = (int(msg[3,3])*1+int(msg[3,4])*2+int(msg[4,4])*4+int(msg[4,3])*8)
    idx>>ori//90|((idx<<(4-(ori//90)))&0b1111)
    return msg,ori,idx

In [8]:
sharpen = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])## Sharpen the image to hightlight edges
img = frames[0] #assign the frame of interest
img = cv.medianBlur(img, 5) # Blur the image slightly
img = cv.filter2D(img,-1,sharpen) ##Sharpen the image to get the edges more promanent 
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY) #convert to grayscale

#Perform corner detection
corners = cv.goodFeaturesToTrack(gray,25,0.35,10) 
corners = np.int0(corners)
list_of_corners = []
for i in corners:
    x,y = i.ravel()
    list_of_corners.append(np.array([x,y]))
    cv.circle(img, (x,y), 3, 255, -1)

In [9]:
## detect all the rectangle in the image
rectangles,d=detect_rectangles(corners)
tags = []
for i, rectangle in enumerate(rectangles):
    #### MAKE IT TO THE CLOSEST MULTIPLE OF 8 the side length of tag
    max_frame_size = int(d[i][1])+(int(d[i][1])%8)
    PoF = [[0,0],[0,max_frame_size],[max_frame_size,max_frame_size],[max_frame_size,0]]
    
    homo = calculate_homography(rectangle,PoF)
    
    warped_img = cv.warpPerspective(frames[0], homo,(max_frame_size,max_frame_size))
    cv.imshow("warped_img",warped_img)
    cv.waitKey()
    cv.destroyAllWindows()
    try:
        msg,ori,idx = decode_tag(warped_img)
    except:
        continue
    tags.append([msg,ori,idx])

In [10]:
tags[0]

[array([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 0., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]),
 90,
 14]